In [22]:
'''importing required lib'''
import numpy as np # to deal with numbers and array
import pandas as pd #for handling dataframes
from sklearn.utils import shuffle #use to shuffle dataframe

''' lib required for creating a model  i.e text data into array of matrix'''
from sklearn.feature_extraction.text import CountVectorizer  ## BOW Model
from sklearn.feature_extraction.text import TfidfVectorizer  ## TFIDF Model
# for converting text data into array 

''' lib required for cleaning of data'''
from nltk.corpus import stopwords #containing stopwords
from nltk.stem.porter import PorterStemmer # use to fing a core meaning of words 
from nltk.stem.wordnet import WordNetLemmatizer
import re #used for regular expression operation
import os
import nltk
import pickle

'''Algorithms used
naivebayes>>>MultinomialNB ,BernoulliNB
linear_model>>>>>Logistic regression ,SGDClassifier, 
SVM>>>>SVC ,LinearSVC ,NuSVC
RandomForest
'''

'Algorithms used\nnaivebayes>>>MultinomialNB ,BernoulliNB\nlinear_model>>>>>Logistic regression ,SGDClassifier, \nSVM>>>>SVC ,LinearSVC ,NuSVC\nRandomForest\n'

In [2]:
dataframe=pd.read_csv('crisis3.csv',encoding = 'unicode_escape')

In [3]:
dataframe.head() #showing first five values

,Unnamed: 0,label,tweet_id,tweet_text
0,0,other_useful_information,'592326564110585856',RT @divyaconnects: Reached #Kathmandu finally!...
1,1,missing_trapped_or_found_people,'592616512642420737',fears for Foreigners missing in Nepal earthqua...
2,2,injured_or_dead_people,'592686635520827393',"RT @ParisBurned: 3,700 people dead is absolute..."
3,3,donation_needs_or_offers_or_volunteering_services,'593301431366635520',Earthquake in Nepal - Please help Kapil #crowd...
4,4,other_useful_information,'592590231519555584',Nepals Slowing Economy Set for Freefall Witho...


In [4]:
del dataframe['Unnamed: 0']

In [5]:
dataframe.shape # describing the size of dataframe

(20624, 3)

In [6]:
dataframe.columns #information about the attributes

Index(['label', 'tweet_id', 'tweet_text'], dtype='object')

In [7]:
dataframe.describe() # describe the dataframe internal information

,label,tweet_id,tweet_text
count,20624,20624,20624
unique,15,18743,18743
top,other_useful_information,'383770753369264129',RT @AnasMallick: Day3 &amp; CM Balochistan sti...
freq,6231,2,2


In [8]:
dataframe.isnull().sum() #determining whether there is a empty value is present or not

label         0
tweet_id      0
tweet_text    0
dtype: int64

In [9]:
dataframe['label'].unique()

array(['other_useful_information', 'missing_trapped_or_found_people',
       'injured_or_dead_people',
       'donation_needs_or_offers_or_volunteering_services',
       'sympathy_and_emotional_support', 'not_related_or_irrelevant',
       'caution_and_advice', 'displaced_people_and_evacuations',
       'infrastructure_and_utilities_damage', 'prevention',
       'disease_transmission', 'deaths_reports',
       'disease_signs_or_symptoms', 'affected_people', 'treatment'],
      dtype=object)

In [41]:
#other_userful_information=dataframe[dataframe['label']=="other_useful_information"]

In [10]:
def same_class(x):
    if x == "deaths_reports" or x=="injured_or_dead_people" or x=="affected_people":
        return "affectedPeople_or_injured_or_dead_people_or_DeathReports"
    elif x=="disease_transmission" or x=="disease_signs_or_symptoms":
        return "disease_transmission_or_disease_signs_or_symptoms"
    elif x=="displaced_people_and_evacuations" or x=="missing_trapped_or_found_people":
        return 'missing_trapped_or_found_people'
    elif x=="caution_and_advice" or x=="prevention":
        return "prevention_or_caution_and_advice"
    else:
        return x
        
dataframe["label"]=dataframe["label"].map(same_class)

In [11]:
print(dataframe['label'].value_counts()) # number of events with there count

other_useful_information                                    6231
affectedPeople_or_injured_or_dead_people_or_DeathReports    3051
not_related_or_irrelevant                                   2758
donation_needs_or_offers_or_volunteering_services           2288
sympathy_and_emotional_support                              1875
infrastructure_and_utilities_damage                         1336
prevention_or_caution_and_advice                            1282
missing_trapped_or_found_people                              751
disease_transmission_or_disease_signs_or_symptoms            682
treatment                                                    370
Name: label, dtype: int64


In [44]:
#dataframe=dataframe.drop(dataframe[(dataframe['label']=="other_useful_information")].index)

In [45]:
#dataframe[dataframe['label']=="not_related_or_irrelevant"]

In [46]:
#dataframe.to_csv('crisis3.csv')

In [47]:
label1={'other_useful_information':0, 'missing_trapped_or_found_people':1,
       'affectedPeople_or_injured_or_dead_people_or_DeathReports':2,
       'donation_needs_or_offers_or_volunteering_services':3,
       'sympathy_and_emotional_support':4, 'not_related_or_irrelevant':5,
       'prevention_or_caution_and_advice':6,
       'infrastructure_and_utilities_damage':7,
       'disease_transmission_or_disease_signs_or_symptoms':8, 'treatment':9}
def label(x):
    return label1[x];

In [48]:
dataframe["numeric_label"]=dataframe["label"].map(label)

In [12]:
dataframe['label'].value_counts() # number of events with there count 

other_useful_information                                    6231
affectedPeople_or_injured_or_dead_people_or_DeathReports    3051
not_related_or_irrelevant                                   2758
donation_needs_or_offers_or_volunteering_services           2288
sympathy_and_emotional_support                              1875
infrastructure_and_utilities_damage                         1336
prevention_or_caution_and_advice                            1282
missing_trapped_or_found_people                              751
disease_transmission_or_disease_signs_or_symptoms            682
treatment                                                    370
Name: label, dtype: int64

In [50]:
dataframe['numeric_label'].value_counts() # number of events with there count 

2    3051
3    2288
4    1875
7    1336
6    1282
1     751
8     682
9     370
Name: numeric_label, dtype: int64

In [13]:
print(dataframe['tweet_text'][5])
print("event extracted>>>")
print(dataframe['label'][5])
'''tweet with corresponding label'''

RT @cuteinsan: In this horrible situation in nepal our God  @Gurmeetramrahim  can help pita ji they really need your simpathy #MSGHelpEarth
event extracted>>>
sympathy_and_emotional_support


'tweet with corresponding label'

In [14]:
dataframe['tweet_id'].nunique() #number of unique tweet ID

18743

In [15]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'very', 'further', "don't", 'she', 'doesn', 'they', 'if', 'too', 'out', 'same', 'most', "it's", 'itself', 'i', 'him', 'than', 's', 'mustn', 'own', 'but', 'all', 'can', 'am', 'wasn', 'myself', 'with', 'these', 'm', 't', 'who', "couldn't", 'by', "mightn't", "wasn't", 'before', 'where', 're', 'yourself', 'there', 'haven', 'ain', 'was', "you'll", 'aren', 'd', 'after', 'because', 'shan', 'each', 'himself', 'being', 'that', 'some', "hadn't", 'hasn', 'mightn', 'herself', "needn't", 'about', 'shouldn', "won't", 'and', 'once', 'an', "aren't", 'having', 'how', 'until', "isn't", 'in', 'ours', 'doing', 'y', 'again', 'not', 've', 'which', 'or', 'didn', 'yourselves', 'the', "should've", 'at', 'up', 'for', "you've", 'from', 'into', "shouldn't", "that'll", 'are', 'hers', 'when', 'their', 'my', 'do', 'won', "mustn't", 'were', 'few', 'yours', 'theirs', 'while', 'ma', 'through', 'over', 'should', 'why', "wouldn't", 'isn', 'ourselves', 'under', 'its', 'just', 'has', "she's", 'his', 'a', 'whom', 'be', "ha

In [16]:
new_words = ["RT",'rt'] #creating a list of extra stop words
stop = stop_words.union(new_words) #adding words into corpus

In [17]:
'''
Input : The whole dataframe will be send to the function
Output : Column with clean tweets tweet will be added
Function perform following operation:
       1. Clean text from html tags
       2. Clean text from punctuations and special characters
       3. Retain only non-numeric Latin characters with lenght > 2
       4. Remove stopwords from the sentence
       5. Apply stemming to all the words in the sentence
'''
def clean_tweets(dataset):
    corpus = []
    for i in range(0, dataset.shape[0]):
        text=dataset['tweet_text'][i]
        #hte line data.shape will return [row size,columnsize ] from which we are taking row size using index 0
        
        #remove @mentions
        rg=re.compile('@[^, ]*\s*')
        text=re.sub(rg," ",text)

        #remove urls
        rg=re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text=re.sub(rg," ",text)

        #Remove punctuations
        text = re.sub('[^a-zA-Z0-9]', ' ',text)

        #Convert to lowercase
        text = text.lower()

        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

        # remove special characters and digits
        #text=re.sub("(\\d|\\W)+"," ",text)
        text=re.sub("(\\W)+"," ",text)

        ##Convert to list from string
        text = text.split()

        ##Stemming
        ps=PorterStemmer()    #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in
                stop_words and len(word)>2]
        text = " ".join(text)
        corpus.append(text)
    dataset['preprocessed']=corpus
    return dataset

In [18]:
dataframe=clean_tweets(dataframe)# function calling for preprocessed tweet
dataframe.head()

,label,tweet_id,tweet_text,preprocessed
0,other_useful_information,'592326564110585856',RT @divyaconnects: Reached #Kathmandu finally!...,reached kathmandu finally lot indian stranded ...
1,missing_trapped_or_found_people,'592616512642420737',fears for Foreigners missing in Nepal earthqua...,fear foreigner missing nepal earthquake death ...
2,affectedPeople_or_injured_or_dead_people_or_De...,'592686635520827393',"RT @ParisBurned: 3,700 people dead is absolute...",700 people dead absolutely devastating country...
3,donation_needs_or_offers_or_volunteering_services,'593301431366635520',Earthquake in Nepal - Please help Kapil #crowd...,earthquake nepal please help kapil crowdfundin...
4,other_useful_information,'592590231519555584',Nepals Slowing Economy Set for Freefall Witho...,nepal slowing economy set freefall without wor...


In [19]:
dataframe.shape

(20624, 4)

In [21]:
'''
with open('mainprocessed_dataset20624.pickle' ,'wb') as f:
    pickle.dump(dataframe,f)
'''

"\nwith open('mainprocessed_dataset20624.pickle' ,'wb') as f:\n    pickle.dump(dataframe,f)\n"

In [31]:
pickle_in =open('scripts/preprocessed.pickle' ,'rb')
dataframe=pickle.load(pickle_in)

In [32]:
print(dataframe['tweet_text'][2])
print('After preprocessing>>>>>>>>>>>>>>>>>')
print(dataframe['preprocessed'][2])

Earthquake in Nepal - Please help Kapil #crowdfunding fundrazr Support and Retweet http://t.co/mFEyQ85js9 via MechanismOfWar
After preprocessing>>>>>>>>>>>>>>>>>
earthquake nepal please help kapil crowdfunding fundrazr support retweet via mechanismofwar


In [33]:

X = dataframe['preprocessed'].values# text data will be converted into operational format
y=dataframe['label'].values #same as it is'''
#print(X[0:5])
print('corresponding result >>>>>>>>>>>>>>>>>>>>')
print(y[0:5])


corresponding result >>>>>>>>>>>>>>>>>>>>
['missing_trapped_or_found_people'
 'affectedPeople_or_injured_or_dead_people_or_DeathReports'
 'donation_needs_or_offers_or_volunteering_services'
 'sympathy_and_emotional_support'
 'affectedPeople_or_injured_or_dead_people_or_DeathReports']


In [34]:
tf_idf_vect = TfidfVectorizer()# initializing the vector by creating object
X= tf_idf_vect.fit_transform(X) # tranforming text data into vector format
X.shape

(11635, 11452)

In [35]:

with open("tfidf_model_11452.pickle",'wb') as f:
    pickle.dump(tf_idf_vect,f) 


In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 123)

In [38]:
from sklearn import metrics

In [39]:
from sklearn.svm import LinearSVC
linear_svc_model = LinearSVC(multi_class="ovr") # note perform changes in attribute to get bettter accuracy
linear_svc_model.fit(X_train,y_train)#passing data into model for training
y_pred=linear_svc_model.predict(X_test)#after training we pass rest 20%of data for testing purpose
acc=metrics.accuracy_score(y_test,y_pred) 
#here we are passing actual result and predicted result to find out accuracy
print('acuuracy>>>>',acc*100)

acuuracy>>>> 86.08247422680412


In [23]:
with open("linearSvc.pickle" ,'wb') as f:
    pickle.dump(linear_svc_model,f)

In [12]:
'''
from xgboost import XGBClassifier
from time import ctime ,time
print(ctime())
xgbm=XGBClassifier(objective="multi:softmax",max_depth=7,n_estimator=800,learning_rate=0.7)
xgbm.fit(X_train,y_train)
y_pred=xgbm.predict(X_test)
acc=metrics.accuracy_score(y_test,y_pred) 
#here we are passing actual result and predicted result to find out accuracy
print('acuuracy>>>>',acc*100)
print(ctime())
'''

'\nfrom xgboost import XGBClassifier\nfrom time import ctime ,time\nprint(ctime())\nxgbm=XGBClassifier(objective="multi:softmax",max_depth=7,n_estimator=800,learning_rate=0.7)\nxgbm.fit(X_train,y_train)\ny_pred=xgbm.predict(X_test)\nacc=metrics.accuracy_score(y_test,y_pred) \n#here we are passing actual result and predicted result to find out accuracy\nprint(\'acuuracy>>>>\',acc*100)\nprint(ctime())\n'